In [1]:
import os

import numpy as np
import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import Point, mapping, Polygon

from bokeh.layouts import gridplot
from bokeh.models import BoxSelectTool, LassoSelectTool
from bokeh.plotting import figure, show, save, row, ColumnDataSource
from bokeh.io import output_notebook, output_file
from bokeh.models import Band, ColumnDataSource, VArea, HArea

output_notebook()

Loading BokehJS ...

In [2]:
BASE_DIR = os.getcwd()

HYSETS_DIR = os.path.join(BASE_DIR, 'source_data/HYSETS_data/')

source_DEM = 'EENV_DEM' # EENV_DEM or USGS_3DEP

basin_gen_revision = 'RC'

processed_data_path = f'processed_data/basin_properties/{basin_gen_revision}/'

In [3]:
output_folder = os.path.join(BASE_DIR, f'processed_data/plots/{basin_gen_revision}')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [4]:
hysets_df = pd.read_csv(os.path.join(HYSETS_DIR, 'HYSETS_watershed_properties.txt'), sep=';')
hysets_locs = [Point(x, y) for x, y in zip(hysets_df['Centroid_Lon_deg_E'].values, hysets_df['Centroid_Lat_deg_N'])]
hysets_df = gpd.GeoDataFrame(hysets_df, geometry=hysets_locs, crs='EPSG:4326')

In [5]:
hysets_df.columns

Index(['Watershed_ID', 'Source', 'Name', 'Official_ID', 'Centroid_Lat_deg_N',
       'Centroid_Lon_deg_E', 'Drainage_Area_km2', 'Drainage_Area_GSIM_km2',
       'Flag_GSIM_boundaries', 'Flag_Artificial_Boundaries', 'Elevation_m',
       'Slope_deg', 'Gravelius', 'Perimeter', 'Flag_Shape_Extraction',
       'Aspect_deg', 'Flag_Terrain_Extraction', 'Land_Use_Forest_frac',
       'Land_Use_Grass_frac', 'Land_Use_Wetland_frac', 'Land_Use_Water_frac',
       'Land_Use_Urban_frac', 'Land_Use_Shrubs_frac', 'Land_Use_Crops_frac',
       'Land_Use_Snow_Ice_frac', 'Flag_Land_Use_Extraction',
       'Permeability_logk_m2', 'Porosity_frac', 'Flag_Subsoil_Extraction',
       'geometry'],
      dtype='object')

In [6]:
def get_region_polygon(region):
    region_polygon_path = f'processed_data/merged_basin_groups/region_polygons/'
    region_polygon_fpath = os.path.join(BASE_DIR, region_polygon_path)
    region_polygon_fnames = os.listdir(region_polygon_fpath)
    region_polygon_files = [e for e in region_polygon_fnames if e.startswith(region)]
    assert len(region_polygon_files) == 1
    region_polygon_fname = region_polygon_files[0]

    return gpd.read_file(os.path.join(region_polygon_fpath, region_polygon_fname))

In [7]:
# def retrieve_results(region, param, method, data_path):
#     region_polygon = get_region_polygon(region)    
    
#     # basin characteristics derived independently using derived basin polygons
#     sample_fname = f'processed_data/basin_properties/{region}/{region}_basin_properties_{method}_{simulation_no}.csv'
#     sample_fpath = os.path.join(BASE_DIR, sample_fname)
#     sample_df = pd.read_csv(sample_fpath)
#     sample_df = sample_df[[c for c in sample_df.columns if 'Unnamed' not in c]]
    
#     # find all hysets basins falling inside the region polygon
#     polygon = region_polygon.geometry.values[0]
#     assert region_polygon.crs == hysets_df.crs
    
#     hysets_within = hysets_df[hysets_df.within(polygon)]
#     print(f'    There are {len(hysets_within)} HYSETS stations within {region} region.')
            
#     s_data = sample_df[param].values
    
#     h_data = hysets_within[param].values
#     data = {
#         'x1': np.sort(s_data),
#         'y1': np.arange(len(s_data)) / float(len(s_data)),
#         'x2': np.sort(h_data),
#         'y2': np.arange(len(h_data)) / float(len(h_data)),
#     }
#     return data
    

In [8]:
def make_ecdf_plot(region, param, data):
    
    x1, y1 = data['x1'], data['y1'] # population estimate
    x2, y2 = data['x2'], data['y2'] # Hysets stations
    
    n_pts_pop = len(data['x2'])
    n_pts_hysets = len(data['x1'])
    
    p = figure(width=400, height=300, title=f'{region} CDF')
    p.circle(x1, y1, 
             legend_label=f'Pop est. N={n_pts_pop}',
             color='dodgerblue')
    p.circle(x2, y2, 
             legend_label=f'Hysets N={n_pts_hysets}', 
             color='firebrick')
    
    p.legend.location = 'bottom_right'
    p.xaxis.axis_label = f'{param}'
    p.yaxis.axis_label = 'P(X)'
    
    return p
    

In [9]:
# params = ['Drainage_Area_km2', 'Elevation_m', 'Perimeter',
#        'Aspect_deg', 'Gravelius', 'Slope_deg', 'Permeability_logk_m2',
#        'Porosity_frac', 'Land_Use_Forest_frac', 'Land_Use_Shrubs_frac',
#        'Land_Use_Grass_frac', 'Land_Use_Wetland_frac', 'Land_Use_Crops_frac',
#        'Land_Use_Urban_frac', 'Land_Use_Water_frac', 'Land_Use_Snow_Ice_frac']
attributes = ['Drainage_Area_km2', 'Elevation_m', 'Gravelius', 'Perimeter', 'Slope_deg', 'Aspect_deg']

# basin characteristics derived independently using derived basin polygons
processed_regions = sorted([e.split('_')[0] for e in os.listdir(processed_data_path) if not e.endswith('.csv')])
processed_regions

['07G',
 '07O',
 '07U',
 '08A',
 '08B',
 '08C',
 '08D',
 '08E',
 '08F',
 '08G',
 '08H',
 '08N',
 '08O',
 '08P',
 '09A',
 'ERockies',
 'Fraser']

In [10]:
# for param in params:
#     print('')
#     print(f'Processing {param}')
#     plots = []
#     for region in processed_regions:
#         # pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
#         # cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2
#         data = retrieve_results(region, param, method)
#         p1 = make_ecdf_plot(region, param, data)
#         plots.append(p1)
#     grid = gridplot(plots, width=300, height=250, ncols=3)
#     output_path = os.path.join(BASE_DIR, f'processed_data/plots/{param}_CDF_Plots_{method}.html')
#     save(grid, filename=output_path, title=f'{param} CDF')

In [11]:
def results_all_methods(region, param):
    
    data = {}
    
    for method in ['RAND', 'CONF', 'GRAD']:
                
        simulation_path = os.path.join(processed_data_path, f'{region}/')
        all_simulations = sorted([e for e in os.listdir(simulation_path) if method in e])
        
        sim_paths = [os.path.join(simulation_path, e) for e in all_simulations]
        
        all_dfs = []
        
        df = pd.DataFrame()
        for p in sim_paths:
            sim_no = p.split('.')[0].split('_')[-1]
            sample_df = pd.read_csv(p)
            
            df[sim_no] = sample_df[param]
            
        data[method] = df        
    
    return data
    

In [12]:
def method_comparison_plot(region, attribute, data):
    
    n_sims = len(data['RAND'].columns)
    
    p = figure(width=400, height=300)
    
    cs = ['firebrick', 'dodgerblue', 'gold']
    i = 0
    
    for method in ['RAND', 'CONF', 'GRAD']:
        
        df = data[method]
        n_pts = len(df)
        
        dist_df = pd.DataFrame()
        
        sim_cols = list(df.columns)
        
        p_vals = np.linspace(0, 100, 100)
        # p_vals = pd.qcut(
        
        max_len, min_len = -1, 1E9
        
        for c in sim_cols:
            
            d = df[[c]].copy()
            d.dropna(how='any', inplace=True)
            
            n_good_pts = len(d)
            if n_good_pts < min_len:
                min_len = n_good_pts
            if n_good_pts > max_len:
                max_len = n_good_pts
                        
            if attribute in ['Drainage_Area_km2', 'Perimeter']:
                x_label = 'log_' + attribute
                x = np.sort(np.log10(d[c].values))
                x = np.log10(d[c].values)
            else:
                x_label = attribute
                # x = np.sort(d[c].values)
                x = d[c].values
                                       
            # dist_df[c] = np.percentile(x, p_vals)
            # dist_df[c] = pd.qcut(x, 20)
            dist_df[c] = np.percentile(x, p_vals)
            # x_label = attribute
               
               
        dist_df['median'] = dist_df.quantile(q=0.5, axis=1).values
            
        dist_df['lower'] = dist_df.quantile(q=0.05, axis=1).values

        dist_df['upper'] = dist_df.quantile(q=0.95, axis=1).values
        
        dist_df['x'] = p_vals / 100
        
        dist_df = dist_df[['median', 'x', 'lower', 'upper']]

                        
        source = ColumnDataSource(dist_df)
        
        band = HArea(x1='lower', x2='upper', y='x', 
                     fill_color=cs[i], fill_alpha=0.3)
        p.add_glyph(source, band)
        
        p.line(dist_df['median'], dist_df['x'],
               legend_label=f'{method}',
               alpha=1, line_width=3, color=cs[i])
        
        plot_title=f'{region} CDF {n_sims} simulations, N={n_pts}/sim.'
        
        p.title.text = f'{region} CDF N={n_pts}, {n_sims} simulations'
        
        
        i += 1
    
    p.legend.location = 'bottom_right'
    p.xaxis.axis_label = f'{x_label}'
    p.yaxis.axis_label = 'P(X)'
    
    return p

In [13]:
for attribute in attributes:
    print('')
    print(f'Processing {attribute}')
    plots = []
    for region in ['07G']:#processed_regions:
        print('')
        print(region)
        # pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
        # cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2
        data = results_all_methods(region, attribute)
        p1 = method_comparison_plot(region, attribute, data)
        plots.append(p1)
        
    grid = gridplot(plots, width=400, height=300, ncols=3)
    output_path = os.path.join(output_folder, f'{attribute}_CDF_Plots_method_comparison.html')
    output_file(filename=output_path, title=f'{attribute} CDF')
    save(grid, filename=output_path, title=f'{attribute} CDF')


Processing Drainage_Area_km2

07G

07O

07U

08A

08B

08C

08D

08E

08F

08G

08H

08N

08O

08P

09A

ERockies

Fraser


UnboundLocalError: local variable 'x_label' referenced before assignment